In [1]:
from utils.utils import *
from scripts.config import DefaultArgsNamespace
import torch
import torch.nn as nn
import torchvision.models as models
from datautils.ems import *
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR
import wandb


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

args = DefaultArgsNamespace()

# Access the parsed arguments
model, optimizer, criterion, device = init_model(args)# verbose_mode = args.verbose
scheduler = StepLR(optimizer, step_size=args.learning_params["lr_drop"], gamma=0.1)  # adjust parameters as needed

dummy_input = torch.randn(5, 30, 3, 224, 224) # batch, num_frames, channels, height, width

dummy_input = dummy_input.to(args.device)

dummy_output = model(dummy_input)   

print(dummy_output.shape)




/home/cjh9fw/.local/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/sfs/qumulo/qhome/cjh9fw/.local/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/home/cjh9fw/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/cjh9fw/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to 

torch.Size([5, 16])


In [3]:
train_loader, val_loader, test_loader = eee_get_dataloaders(args)

train dataset size:  10
val dataset size:  3
test dataset size:  4


In [3]:

# print one batch
for i, batch in enumerate(train_loader):
    print(batch['frames'].shape, batch['audio'].shape, batch['flow'].shape, batch['rgb'].shape, batch['keystep_label'], batch['keystep_id'], batch['start_frame'], batch['end_frame'],batch['start_t'], batch['end_t'],  batch['subject_id'], batch['trial_id'])

    break

# # Train the model
# for epoch in range(args.learning_params["epochs"]):
#     train_loss = train_one_epoch(model, train_loader, criterion, optimizer, device, wandb_logger)
#     val_loss = validate(model, val_loader, criterion, device, wandb_logger)
#     scheduler.step()
#     print(f"Epoch: {epoch}, Train Loss: {train_loss}, Val Loss: {val_loss}")


/home/cjh9fw/.local/lib/python3.8/site-packages/torchvision/io/video_reader.py:245: UserWarning: Accurate seek is not implemented for pyav backend
  warnings.warn("Accurate seek is not implemented for pyav backend")
/home/cjh9fw/.local/lib/python3.8/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


torch.Size([1, 274, 3, 224, 224]) torch.Size([1, 422912, 2]) torch.Size([1, 264, 1024]) torch.Size([1, 264, 1024]) ['administer_shock_aed'] tensor([7]) tensor([2382]) tensor([2646]) tensor([79.4036], dtype=torch.float64) tensor([88.1953], dtype=torch.float64) ['ng1'] ['1']
